# Check input data produced by Python against the one produced by R
data have been pickled at the end of preprocess before using in STAN model

R and Python generating scripts should be launched before

R: `Rscript dump-input-base-region-france.r --full` 

Python : `python Python/src/run-model.py --full --only-dump-input`




In [ ]:
import pickle
import pyreadr
import pandas as pd
import numpy as np

## load R data

In [ ]:
with open("../../model_input_R.pkl", "rb") as fd:
    objR = pickle.load(fd)

## load Python data

In [ ]:
with open("../../model_input_Py.pkl", "rb") as fd:
    objPy = pickle.load(fd)

In [ ]:
PyKeys = set(objPy.keys())
RKeys = set(objR.keys())

In [ ]:
ok_fields = set(['deaths', 'cases', 'covariate1', 'covariate2', 'covariate3', 'covariate5', 'covariate6', 'EpidemicStart',
'N', 'N0', 'N2', 'pop', 'x',
'M', #  => check why missing 4 regions
'regions'
])

In [ ]:
# find which row matches between Py and R looking into deaths

for field in ['cases', 'deaths']:
    correspondance = {}
    for idx, row_death in enumerate(objPy[field].transpose()):
        results = []
        for idxR, row_deathR in enumerate(objR[field].transpose()):
            if (row_death == row_deathR).all():
                results.append(idxR)
        correspondance[idx] = results
    print(field, correspondance)

In [ ]:
objPy[field].shape, objR[field].shape

In [ ]:
sel = (objPy[field].transpose()[0] == objR[field].transpose()[9]) == False
objPy[field].transpose()[0][sel],  objR[field].transpose()[9][sel]


In [ ]:
for idx, row in enumerate(objR[field].transpose()):
    if 20 in row:
        if 11 in row:
            if 12 in row:
                print(idx, row)


In [ ]:
map_Py_R = {k: v[0] for k, v in correspondance.items()}
map_R_Py = {v[0]: k for k, v in correspondance.items()}


In [ ]:
for field in RKeys:
    if field not in ['covariate4']: 
        row_Py = objPy[field]
        row_R = objR[field]
        if isinstance(row_Py, (int, float)):
            print(field, type(row_Py), row_Py == row_R)
        elif isinstance(row_Py, (list, set)):
            if len(row_Py) == len(map_Py_R):
                same = True
                for idxPy, idxR in map_Py_R.items():
                    same &= row_Py[idxPy] == row_R[idxR]
                print(field, same)
            else:
                print(field, type(row_Py), row_Py == row_R)
        elif hasattr(row_Py, 'shape'):
            if row_Py.shape[0] == len(map_Py_R):
                same = True
                for idxPy, idxR in map_Py_R.items():
                    same &= row_Py[idxPy] == row_R[idxR]
                print(field, same)
            elif row_Py.shape[1] == len(map_Py_R):
                print(field)
                same = True
                for idxPy, idxR in map_Py_R.items():
                    try:
                        same &= row_Py[:,idxPy] == np.array(row_R[:,idxR])
                        #same &= row_Py[:,idxPy] == row_R[:,idxR]
                    except:
                        same &= row_Py[:,idxPy] == np.array(row_R[:,idxR])
                print(field, same)


In [ ]:
print([(key, objPy[key].shape, objR[key].shape) for key in PyKeys if hasattr(objPy[key],'shape')])

In [ ]:
field = 'f'
print(objPy[field].shape, objR[field].shape)
print((objPy[field] != objR[field]).sum())


In [ ]:
PyKeys.difference(ok_fields)

In [ ]:
print(objPy['deaths'].shape, objR['deaths'].shape)
print(sum(objPy['deaths'] < 0), sum(objR['deaths'] < 0))

In [ ]:
field = 'f'
objPy[field].shape , objR[field].shape
#np.array(objPy[field]) == np.array(objR[field])


In [ ]:
#objPy['regions'], objR['regions']

In [ ]:
set(objPy['regions']).difference(set(objR['regions'])), set(objR['regions']).difference(set(objPy['regions']))


In [ ]:
#field = 'M'
#objPy[field] , objR[field]

In [ ]:
#PyKeys

In [ ]:
idxPyItaly = objPy['regions'].index('Italy')
idxRItaly = objR['regions'].index('Italy')
print(idxPyItaly, idxRItaly)

In [ ]:
#len(objPy['deaths'][:,idxPyItaly]),len(objR['deaths'][:,idxRItaly])
idx_t = 80
objPy['deaths'][:,idxPyItaly][idx_t:idx_t+10],objR['deaths'][:,idxRItaly][idx_t:idx_t+10]


In [ ]:
print(PyKeys.difference(RKeys), RKeys.difference(PyKeys))

In [ ]:
print([(key, type(objPy[key]), type(objR[key])) for key in PyKeys if type(objPy[key]) != type(objR[key])])

In [ ]:
idxPy = 0
idxR = map_Py_R[idxPy]

# covariat4 ???? => try values
covariate_names = ['covariate{}'.format(idx+1) for idx in range(6) if idx != 3]
covariate_names = ['covariate{}'.format(idx+1) for idx in range(6)]

cov_corr = {}
for fieldPy in covariate_names:
    results = []
    #print('py', fieldPy)
    row_Py = objPy[fieldPy][:,idxPy] 
    for fieldR in covariate_names:
        #print('R', fieldR, objR[fieldR].shape)
        try:
            row_R = objR[fieldR][:,idxR] 
        except:
            row_R = objR[fieldR].values[:,idxR] 
        if (row_Py == row_R).all():
            results.append(fieldR)
    cov_corr[fieldPy] = results

print(cov_corr)



In [ ]:
len(objPy[field])

In [ ]:
idx_t = 45
#field = 'cases'
field = 'EpidemicStart'
field = 'N'
field = 'pop'

#
print(objPy[field][idxPy] == objR[field][idxR])
print(objPy[field][idxPy], objR[field][idxR])


In [ ]:
idx_t = 45
#field = 'cases'
field = 'covariate1'
#
print(objPy[field][:,idxPy].astype(np.int) == objR[field][:,idxR].astype(np.int))
idx_t = 12
print(objPy[field][:,idxPy][idx_t:idx_t+10],objR[field][:,idxR][idx_t:idx_t+10])
idx_t = 45
print(objPy[field][:,idxPy][idx_t:idx_t+10],objR[field][:,idxR][idx_t:idx_t+10])


In [ ]:
for idxPy, lidxR in correspondance.items():
    for idxR in lidxR:
        print(objPy['regions'][idxPy], objR['regions'][idxR])

In [ ]:
print([el for el in objPy['regions'] if 'grand' in el.lower()])
idxPyGrand = objPy['regions'].index('Grand Est')
idxRGrand = objR['regions'].index('Grand Est')

#len(objPy['deaths'][:,idxPyItaly]),len(objR['deaths'][:,idxRItaly])
idx_t = 10

objPy['deaths'][:,idxPyGrand][idx_t:idx_t+10],objR['deaths'][:,idxRGrand][idx_t:idx_t+10]

#objPy['deaths'][:,idxPyGrand] == objR['deaths'][:,idxRGrand]

In [ ]:
print(correspondance)

In [ ]:
#print([(key, objPy[key] == objR[key]) for key in PyKeys])
print([(key, (objPy[key] == objR[key]).all()) for key in PyKeys if hasattr(objPy[key],'shape')])

In [ ]:
df_npi = pd.read_csv("data/interventions.csv")
df_npi.head(3)

In [ ]:
df_interventions = df_npi.copy()

# transpose
#df_interventions = df_interventions.reset_index().groupby('Country')
#[['Country', 'Event', 'Date effective']]\
df_interventions = df_interventions\
    .groupby('Country')[['Event', 'Date effective']].apply(lambda df: df.reset_index(drop=True)
    .groupby('Event').apply(lambda df: df[['Date effective']].reset_index(drop=True)).unstack()  #.apply(lambda df: df.reset_index(drop=True)) #.unstack())
    .transpose()) #.transpose().unstack()
#df.groupby('userid')['name'].apply(lambda df: df.reset_index(drop=True)).unstack()
#print(df_interventions.columns)
#df_interventions = df_interventions.set_index
#    .transpose()
df_interventions = df_interventions.reset_index().set_index('Country')
df_interventions.drop(columns=['level_1', 'level_2'], inplace=True)


In [ ]:
df_interventions.head()

In [ ]:
data_france_R = pyreadr.read_r("data/all-france.rds")

In [ ]:
df_france_formated = pd.read_csv("data/all-france_formated.csv")

df_france_formated[df_france_formated.countriesAndTerritories == "Grand Est"].head()


In [ ]:
df_france = pd.read_csv("data/all-france.csv")

In [ ]:
df_france[df_france.countriesAndTerritories == "Grand Est"].head()

In [ ]:
df_france.countriesAndTerritories.unique() # .head()

In [ ]:
data_france_R